# Clean - Clean Dataset
This example illustrate how to perform meta-blocking on a clean-clean dataset (data linkage), so when we have two datasets that do not contain duplicates and we want to discover the duplicates between them. 

In [1]:
import sparker
import random
import pandas as pd

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('SparkByExamples.com').getOrCreate()

2022-04-11 22:31:45,746 WARN util.Utils: Your hostname, aditi-OMEN-by-HP-Laptop-16-b0xxx resolves to a loopback address: 127.0.1.1; using 192.168.0.115 instead (on interface wlo1)
2022-04-11 22:31:45,747 WARN util.Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
2022-04-11 22:31:46,136 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
2022-04-11 22:31:46,607 WARN util.Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
2022-04-11 22:31:46,607 WARN util.Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


## Load the data
sparkER provides wrappers to load CSV and JSON files.

First, we load the first dataset, and we extract the maximum id. The profiles ids of the second dataset will be assigned starting from this one.

*real_id_field* is the field that contains the identifier of the record.

In [3]:
# Profiles contained in the first dataset
profiles1 = sparker.CSVWrapper.load_profiles(file_path = '/Final_Project/sample/Sample1_cleaned.csv',
                                             start_id_from = 0,
                                             separator=",", header=True,
                                             real_id_field = "id",
                                            source_id = 0)
# Max profile id in the first dataset, used to separate the profiles in the next phases
separator_id = profiles1.map(lambda profile: profile.profile_id).max()
# Separators, used during blocking to understand from which dataset a profile belongs. It is an array because sparkER
# could work with multiple datasets
separator_ids = [separator_id]

In [4]:
type(separator_id)

int

In [5]:
separator_id

29

In [6]:
profiles1.take(5)
profiles1.toDF().show(1364)

+--------------------+-----------+----------+---------+
|          attributes|original_id|profile_id|source_id|
+--------------------+-----------+----------+---------+
|[{DisplayName, Co...|          0|         0|        0|
|[{DisplayName, Ge...|          1|         1|        0|
|[{DisplayName, Ji...|          2|         2|        0|
|[{DisplayName, St...|          3|         3|        0|
|[{DisplayName, tx...|          4|         4|        0|
|[{DisplayName, Ke...|          5|         5|        0|
|[{DisplayName, Em...|          6|         6|        0|
|[{DisplayName, To...|          7|         7|        0|
|[{DisplayName, Se...|          8|         8|        0|
|[{DisplayName, Fr...|          9|         9|        0|
|[{DisplayName, Br...|         10|        10|        0|
|[{DisplayName, cm...|         11|        11|        0|
|[{DisplayName, kā...|         12|        12|        0|
|[{DisplayName, Sc...|         13|        13|        0|
|[{DisplayName, Ma...|         14|        14|   

Let's visualize a profile to check if they are correctly loaded

In [7]:
print(profiles1.take(1)[0])

{'profile_id': 0, 'attributes': [{'key': 'DisplayName', 'value': 'Community'}, {'key': 'Location', 'value': 'on the server farm'}, {'key': 'AboutMe', 'value': "Hi, I'm not really a person. I'm a background process that helps keep this site clean! I do things like Randomly poke old unanswered questions every hour so they get some attention. Own community questions and answers so nobody gets unnecessary reputation from them. Own downvotes on spam/evil posts that get permanently deleted. Own suggested edits from anonymous users. Remove abandoned questions."}, {'key': 'Age', 'value': '27'}, {'key': 'phone', 'value': '655-452-4816'}], 'original_id': '0', 'source_id': 0}


Loads the second dataset and extract the max id (it will be used later)

In [8]:
profiles2 = sparker.CSVWrapper.load_profiles('file:///home/aditi/BigData/AAI_Project/dataset/Sample2_cleaned.csv', 
                                              start_id_from = separator_id+1, 
                                              real_id_field = "id",
                                            source_id = 1)
# Max profile id
max_profile_id = profiles2.map(lambda profile: profile.profile_id).max()

In [9]:
max_profile_id

59

In [10]:
profiles2.count()

30

In [11]:
profiles2.toDF().show(3226)

+--------------------+-----------+----------+---------+
|          attributes|original_id|profile_id|source_id|
+--------------------+-----------+----------+---------+
|[{DisplayName, ta...|          0|        30|        1|
|[{DisplayName, Al...|          1|        31|        1|
|[{DisplayName, ja...|          2|        32|        1|
|[{DisplayName, fw...|          3|        33|        1|
|[{DisplayName, Ce...|          4|        34|        1|
|[{DisplayName, Be...|          5|        35|        1|
|[{DisplayName, Ji...|          6|        36|        1|
|[{DisplayName, sa...|          7|        37|        1|
|[{DisplayName, ja...|          8|        38|        1|
|[{DisplayName, Da...|          9|        39|        1|
|[{DisplayName, Pa...|         10|        40|        1|
|[{DisplayName, va...|         11|        41|        1|
|[{DisplayName, Ph...|         12|        42|        1|
|[{DisplayName, Av...|         13|        43|        1|
|[{DisplayName, He...|         14|        44|   

Finally, concatenate the two RDDs

In [12]:
profiles = profiles1.union(profiles2)

In [13]:
profiles.take(2)

[{'profile_id': 0, 'attributes': [{'key': 'DisplayName', 'value': 'Community'}, {'key': 'Location', 'value': 'on the server farm'}, {'key': 'AboutMe', 'value': "Hi, I'm not really a person. I'm a background process that helps keep this site clean! I do things like Randomly poke old unanswered questions every hour so they get some attention. Own community questions and answers so nobody gets unnecessary reputation from them. Own downvotes on spam/evil posts that get permanently deleted. Own suggested edits from anonymous users. Remove abandoned questions."}, {'key': 'Age', 'value': '27'}, {'key': 'phone', 'value': '655-452-4816'}], 'original_id': '0', 'source_id': 0},
 {'profile_id': 1, 'attributes': [{'key': 'DisplayName', 'value': 'Geoff Dalgas'}, {'key': 'Location', 'value': 'Corvallis, OR'}, {'key': 'AboutMe', 'value': 'Dev #2 who helped create Stack Overflow currently working at Microsoft. I love all people willing to share their knowledge. Without community we are nothing. Find me

In [14]:
profiles.count()

60

In [15]:
profiles.toDF().show(4590) #b000jz4hqo

+--------------------+-----------+----------+---------+
|          attributes|original_id|profile_id|source_id|
+--------------------+-----------+----------+---------+
|[{DisplayName, Co...|          0|         0|        0|
|[{DisplayName, Ge...|          1|         1|        0|
|[{DisplayName, Ji...|          2|         2|        0|
|[{DisplayName, St...|          3|         3|        0|
|[{DisplayName, tx...|          4|         4|        0|
|[{DisplayName, Ke...|          5|         5|        0|
|[{DisplayName, Em...|          6|         6|        0|
|[{DisplayName, To...|          7|         7|        0|
|[{DisplayName, Se...|          8|         8|        0|
|[{DisplayName, Fr...|          9|         9|        0|
|[{DisplayName, Br...|         10|        10|        0|
|[{DisplayName, cm...|         11|        11|        0|
|[{DisplayName, kā...|         12|        12|        0|
|[{DisplayName, Sc...|         13|        13|        0|
|[{DisplayName, Ma...|         14|        14|   

In [16]:
newColumns = ["attributes","original_id","profile_id","source_id"]
p = spark.createDataFrame(data = profiles, schema = newColumns)

In [17]:
p.show()

+--------------------+-----------+----------+---------+
|          attributes|original_id|profile_id|source_id|
+--------------------+-----------+----------+---------+
|[{DisplayName, Co...|          0|         0|        0|
|[{DisplayName, Ge...|          1|         1|        0|
|[{DisplayName, Ji...|          2|         2|        0|
|[{DisplayName, St...|          3|         3|        0|
|[{DisplayName, tx...|          4|         4|        0|
|[{DisplayName, Ke...|          5|         5|        0|
|[{DisplayName, Em...|          6|         6|        0|
|[{DisplayName, To...|          7|         7|        0|
|[{DisplayName, Se...|          8|         8|        0|
|[{DisplayName, Fr...|          9|         9|        0|
|[{DisplayName, Br...|         10|        10|        0|
|[{DisplayName, cm...|         11|        11|        0|
|[{DisplayName, kā...|         12|        12|        0|
|[{DisplayName, Sc...|         13|        13|        0|
|[{DisplayName, Ma...|         14|        14|   

In [18]:
tokens_per_profile = profiles.map(lambda profile: (
            profile.attributes))

In [19]:
#tokens_per_profile.take(4)

### Groundtruth (optional)
If you have a groundtruth you can measure the performance of each step.

When you load the groundtruth it contains the original profiles IDs, it is necessary to convert it to use the IDs assigned to each profile by Spark.

In [20]:
# Loads the groundtruth, takes as input the path of the file and the names of the attributes that represent
# respectively the id of profiles of the first dataset and the id of profiles of the second dataset
gt = sparker.CSVWrapper.load_groundtruth('/Final_Project/sample/ground_truth_sample.csv',
                                        'id1', 'id2')

In [21]:
gt.toDF().show(1400)

+---------------+----------------+
|first_entity_id|second_entity_id|
+---------------+----------------+
|             20|               0|
|             21|               1|
|             22|               2|
|             23|               3|
|             24|               4|
|             25|               5|
|             26|               6|
|             27|               7|
|             28|               8|
|             29|               9|
+---------------+----------------+



In [22]:
# Converts the groundtruth by replacing original IDs with those given by Spark
new_gt = sparker.Converters.convert_groundtruth(gt, profiles1, profiles2)

In [23]:
new_gt

{(20, 30),
 (21, 31),
 (22, 32),
 (23, 33),
 (24, 34),
 (25, 35),
 (26, 36),
 (27, 37),
 (28, 38),
 (29, 39)}

In [24]:
len(new_gt)

10

In [25]:
# We can explore some pairs
random.sample(new_gt, 5)

/tmp/ipykernel_22394/3641384983.py:2: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  random.sample(new_gt, 5)


[(28, 38), (26, 36), (29, 39), (21, 31), (20, 30)]

## Blocking
Now we can perform blocking.

By default sparkER performs token blocking, but it is possible to provide a different blocking function.

In the following example each token is splitted in ngrams of size 4 that are used for blocking.

In [26]:
blocks = sparker.Blocking.create_blocks(profiles, separator_ids, 
                                        blocking_method=sparker.BlockingKeysStrategies.ngrams_blocking,
                                        ngram_size=4)
print("Number of blocks",blocks.count())

Number of blocks 1622


In [27]:
sparker.BlockingKeysStrategies.ngrams_blocking

<function sparker.blocking_strategies.BlockingKeysStrategies.ngrams_blocking(attributes, keys_to_exclude=None, ngram_size=3)>

In [28]:
blocks.take(500)

[{'block_id': 0, 'profiles': [{0, 2, 3, 4, 5, 6, 8, 10, 11, 13, 14, 17, 18, 19, 21, 23, 24, 25, 28, 29}, {33, 34, 38, 39, 43, 44, 46, 47, 48, 50, 51, 53, 54, 55, 56, 57, 58, 31}], 'entropy': -1.0, 'cluster_id': -1, 'blocking_key': ''},
 {'block_id': 1, 'profiles': [{0, 1, 3, 5, 8, 10}, {41, 42, 44}], 'entropy': -1.0, 'cluster_id': -1, 'blocking_key': ''},
 {'block_id': 2, 'profiles': [{0}, {56}], 'entropy': -1.0, 'cluster_id': -1, 'blocking_key': ''},
 {'block_id': 3, 'profiles': [{0, 1, 3, 5, 8, 14, 18, 24, 25, 26}, {34, 35, 36, 41, 47, 57}], 'entropy': -1.0, 'cluster_id': -1, 'blocking_key': ''},
 {'block_id': 4, 'profiles': [{0, 1, 2, 3, 5, 8, 10, 12, 14, 15, 17, 18, 19, 21, 24}, {34, 35, 41, 42, 43, 46, 48, 51, 52, 55, 57, 58, 31}], 'entropy': -1.0, 'cluster_id': -1, 'blocking_key': ''},
 {'block_id': 5, 'profiles': [{0, 26, 14}, {43, 36}], 'entropy': -1.0, 'cluster_id': -1, 'blocking_key': ''},
 {'block_id': 6, 'profiles': [{0, 5, 8, 11, 28}, {57, 51, 38}], 'entropy': -1.0, 'clust

Let's continue by using the standard token blocking

In [29]:
# blocks = sparker.Blocking.create_blocks(profiles, separator_ids)
# print("Number of blocks",blocks.count())

In [30]:
blocks.take(500)

[{'block_id': 0, 'profiles': [{0, 2, 3, 4, 5, 6, 8, 10, 11, 13, 14, 17, 18, 19, 21, 23, 24, 25, 28, 29}, {33, 34, 38, 39, 43, 44, 46, 47, 48, 50, 51, 53, 54, 55, 56, 57, 58, 31}], 'entropy': -1.0, 'cluster_id': -1, 'blocking_key': ''},
 {'block_id': 1, 'profiles': [{0, 1, 3, 5, 8, 10}, {41, 42, 44}], 'entropy': -1.0, 'cluster_id': -1, 'blocking_key': ''},
 {'block_id': 2, 'profiles': [{0}, {56}], 'entropy': -1.0, 'cluster_id': -1, 'blocking_key': ''},
 {'block_id': 3, 'profiles': [{0, 1, 3, 5, 8, 14, 18, 24, 25, 26}, {34, 35, 36, 41, 47, 57}], 'entropy': -1.0, 'cluster_id': -1, 'blocking_key': ''},
 {'block_id': 4, 'profiles': [{0, 1, 2, 3, 5, 8, 10, 12, 14, 15, 17, 18, 19, 21, 24}, {34, 35, 41, 42, 43, 46, 48, 51, 52, 55, 57, 58, 31}], 'entropy': -1.0, 'cluster_id': -1, 'blocking_key': ''},
 {'block_id': 5, 'profiles': [{0, 26, 14}, {43, 36}], 'entropy': -1.0, 'cluster_id': -1, 'blocking_key': ''},
 {'block_id': 6, 'profiles': [{0, 5, 8, 11, 28}, {57, 51, 38}], 'entropy': -1.0, 'clust

In [31]:
blocks.take(1)

[{'block_id': 0, 'profiles': [{0, 2, 3, 4, 5, 6, 8, 10, 11, 13, 14, 17, 18, 19, 21, 23, 24, 25, 28, 29}, {33, 34, 38, 39, 43, 44, 46, 47, 48, 50, 51, 53, 54, 55, 56, 57, 58, 31}], 'entropy': -1.0, 'cluster_id': -1, 'blocking_key': ''}]

## Block cleaning

sparkER implements two block cleaning strategies:

* Block purging: discard the largest blocks that involve too many comparisons, the parameter must be >= 1. A lower value mean a more aggressive purging.
* Block cleaning: removes for every profile the largest blocks in which it appears. The parameter is in range ]0, 1\[. A lower value mean a more aggressive cleaning.

In [32]:
# Perfoms the purging
blocks_purged = sparker.BlockPurging.block_purging(blocks, 1.025)

In [33]:
blocks_purged.take(500)

[{'block_id': 0, 'profiles': [{0, 2, 3, 4, 5, 6, 8, 10, 11, 13, 14, 17, 18, 19, 21, 23, 24, 25, 28, 29}, {33, 34, 38, 39, 43, 44, 46, 47, 48, 50, 51, 53, 54, 55, 56, 57, 58, 31}], 'entropy': -1.0, 'cluster_id': -1, 'blocking_key': ''},
 {'block_id': 1, 'profiles': [{0, 1, 3, 5, 8, 10}, {41, 42, 44}], 'entropy': -1.0, 'cluster_id': -1, 'blocking_key': ''},
 {'block_id': 2, 'profiles': [{0}, {56}], 'entropy': -1.0, 'cluster_id': -1, 'blocking_key': ''},
 {'block_id': 3, 'profiles': [{0, 1, 3, 5, 8, 14, 18, 24, 25, 26}, {34, 35, 36, 41, 47, 57}], 'entropy': -1.0, 'cluster_id': -1, 'blocking_key': ''},
 {'block_id': 4, 'profiles': [{0, 1, 2, 3, 5, 8, 10, 12, 14, 15, 17, 18, 19, 21, 24}, {34, 35, 41, 42, 43, 46, 48, 51, 52, 55, 57, 58, 31}], 'entropy': -1.0, 'cluster_id': -1, 'blocking_key': ''},
 {'block_id': 5, 'profiles': [{0, 26, 14}, {43, 36}], 'entropy': -1.0, 'cluster_id': -1, 'blocking_key': ''},
 {'block_id': 6, 'profiles': [{0, 5, 8, 11, 28}, {57, 51, 38}], 'entropy': -1.0, 'clust

In [34]:
# Performs the cleaning
(profile_blocks, profile_blocks_filtered, blocks_after_filtering) = sparker.BlockFiltering.block_filtering_quick(blocks_purged, 0.8, separator_ids)

In [35]:
blocks_purged.take(500)

[{'block_id': 0, 'profiles': [{0, 2, 3, 4, 5, 6, 8, 10, 11, 13, 14, 17, 18, 19, 21, 23, 24, 25, 28, 29}, {33, 34, 38, 39, 43, 44, 46, 47, 48, 50, 51, 53, 54, 55, 56, 57, 58, 31}], 'entropy': -1.0, 'cluster_id': -1, 'blocking_key': ''},
 {'block_id': 1, 'profiles': [{0, 1, 3, 5, 8, 10}, {41, 42, 44}], 'entropy': -1.0, 'cluster_id': -1, 'blocking_key': ''},
 {'block_id': 2, 'profiles': [{0}, {56}], 'entropy': -1.0, 'cluster_id': -1, 'blocking_key': ''},
 {'block_id': 3, 'profiles': [{0, 1, 3, 5, 8, 14, 18, 24, 25, 26}, {34, 35, 36, 41, 47, 57}], 'entropy': -1.0, 'cluster_id': -1, 'blocking_key': ''},
 {'block_id': 4, 'profiles': [{0, 1, 2, 3, 5, 8, 10, 12, 14, 15, 17, 18, 19, 21, 24}, {34, 35, 41, 42, 43, 46, 48, 51, 52, 55, 57, 58, 31}], 'entropy': -1.0, 'cluster_id': -1, 'blocking_key': ''},
 {'block_id': 5, 'profiles': [{0, 26, 14}, {43, 36}], 'entropy': -1.0, 'cluster_id': -1, 'blocking_key': ''},
 {'block_id': 6, 'profiles': [{0, 5, 8, 11, 28}, {57, 51, 38}], 'entropy': -1.0, 'clust

In [36]:
print("Number of blocks_purged",blocks_purged.count())

Number of blocks_purged 1622


If you have the groundtruth, after every blocking step it is possible to check which are the performance of the blocking collection.

In [37]:
recall, precision, cmp_n = sparker.Utils.get_statistics(blocks_after_filtering, max_profile_id, new_gt, separator_ids)

print("Recall", recall)
print("Precision", precision)
print("Number of comparisons", cmp_n)

Recall 1.0
Precision 0.01122334455667789
Number of comparisons 891


## Meta-blocking
Meta-blocking can be used to further refine the block collection removing superfluous comparisons.

SparkER implements different kind of meta-blocking algorithms, you can find the descriptions in our paper.


For every partition of the RDD the pruning algorithm returns as output a triplet that contains:

* The number of edges
* The number of matches (only if the groundtruth is provided)
* The retained edges

To perform the meta-blocking first some data structures have to be created.

In [38]:
from pyspark import SparkContext
sc = SparkContext.getOrCreate()

In [39]:
block_index_map = blocks_after_filtering.map(lambda b : (b.block_id, b.profiles)).collectAsMap()
block_index = sc.broadcast(block_index_map)

# This is only needed for certain weight measures
profile_blocks_size_index = sc.broadcast(profile_blocks_filtered.map(lambda pb : (pb.profile_id, len(pb.blocks))).collectAsMap())

# Broadcasted groundtruth
gt_broadcast = sc.broadcast(new_gt)

### Weighted Node Pruning

In [40]:
WNP_results = sparker.WNP.wnp(
                          profile_blocks_filtered,
                          block_index,
                          max_profile_id,
                          separator_ids,
                          weight_type=sparker.WeightTypes.CBS,
                          groundtruth=gt_broadcast,
                          profile_blocks_size_index=profile_blocks_size_index,
                          comparison_type=sparker.ComparisonTypes.OR
                         )

num_edges = WNP_results.map(lambda x: x[0]).sum()
num_matches = WNP_results.map(lambda x: x[1]).sum()
print("Number of matches", num_matches)
recall = num_matches/len(new_gt)
precision = (num_matches/num_edges)
print("Recall", recall)
print("Precision", precision)
print("Number of comparisons",num_edges)
wnp_f1 = 2*((precision*recall)/(precision+recall))
print("F1 score",wnp_f1)

print("Number of entity resolved",num_matches)

Number of matches 10
Recall 1.0
Precision 0.02004008016032064
Number of comparisons 499
F1 score 0.03929273084479371
Number of entity resolved 10


In [41]:
num_matches

10

The number of edges
The number of matches (only if the groundtruth is provided)
The retained edges

### Reciprocal Weighted Node Pruning

In [42]:
RWNP_results = sparker.WNP.wnp(
                          profile_blocks_filtered,
                          block_index,
                          max_profile_id,
                          separator_ids,
                          weight_type=sparker.WeightTypes.CBS,
                          groundtruth=gt_broadcast,
                          profile_blocks_size_index=profile_blocks_size_index,
                          comparison_type=sparker.ComparisonTypes.AND
                         )
num_edges = RWNP_results.map(lambda x: x[0]).sum()
num_matches = RWNP_results.map(lambda x: x[1]).sum()

recall = num_matches/len(new_gt)
precision = (num_matches/num_edges)
print("Recall", recall)
print("Precision", precision)
print("Number of comparisons",num_edges)
rwnp_f1 = 2*((precision*recall)/(precision+recall))
print("F1 score",rwnp_f1)

Recall 1.0
Precision 0.053763440860215055
Number of comparisons 186
F1 score 0.10204081632653061


In [43]:
newColumns = ["no_of_edges","no_of_matches","retaind_edges"]
RWNP_results.toDF(newColumns).show(10)

+-----------+-------------+--------------------+
|no_of_edges|no_of_matches|       retaind_edges|
+-----------+-------------+--------------------+
|         10|            0|[{0, 55, 19}, {0,...|
|          5|            0|[{2, 58, 15}, {2,...|
|          0|            0|                  []|
|          2|            0|[{6, 45, 12}, {6,...|
|         13|            0|[{8, 35, 47}, {8,...|
|         12|            0|[{10, 57, 44}, {1...|
|         11|            0|[{14, 58, 22}, {1...|
|          2|            0|[{18, 56, 17}, {1...|
|          7|            1|[{24, 57, 32}, {2...|
|          7|            1|[{28, 58, 38}, {2...|
+-----------+-------------+--------------------+
only showing top 10 rows



### Weighted Edge Pruning

In [44]:
WEP_results = sparker.WEP.wep(
                          profile_blocks_filtered,
                          block_index,
                          max_profile_id,
                          separator_ids,
                          weight_type=sparker.WeightTypes.CBS,
                          groundtruth=gt_broadcast,
                          profile_blocks_size_index=profile_blocks_size_index
                         )
num_edges = WEP_results.map(lambda x: x[0]).sum()
num_matches = WEP_results.map(lambda x: x[1]).sum()

recall = num_matches/len(new_gt)
precision = (num_matches/num_edges)
print("Recall", recall)
print("Precision", precision)
print("Number of comparisons",num_edges)
wep_f1 = 2*((precision*recall)/(precision+recall))
print("F1 score",wep_f1)


Recall 1.0
Precision 0.03676470588235294
Number of comparisons 272
F1 score 0.07092198581560283


In [45]:



newColumns = ["no_of_edges","no_of_matches","retaind_edges"]
WEP_results.toDF(newColumns).show(10)

+-----------+-------------+--------------------+
|no_of_edges|no_of_matches|       retaind_edges|
+-----------+-------------+--------------------+
|         13|            0|[{0, 41, 48}, {0,...|
|          4|            0|[{2, 43, 16}, {2,...|
|          0|            0|                  []|
|          1|            0|       [{6, 46, 16}]|
|         17|            0|[{8, 42, 48}, {8,...|
|         19|            0|[{10, 34, 65}, {1...|
|         13|            0|[{14, 43, 54}, {1...|
|          1|            0|      [{18, 56, 17}]|
|         12|            1|[{24, 34, 369}, {...|
|         11|            1|[{28, 36, 16}, {2...|
+-----------+-------------+--------------------+
only showing top 10 rows



### Cardinality Edge Pruning

In [46]:
CEP_results = sparker.CEP.cep(
                          profile_blocks_filtered,
                          block_index,
                          max_profile_id,
                          separator_ids,
                          weight_type=sparker.WeightTypes.CBS,
                          groundtruth=gt_broadcast,
                          profile_blocks_size_index=profile_blocks_size_index
                         )
num_edges = CEP_results.map(lambda x: x[0]).sum()
num_matches = CEP_results.map(lambda x: x[1]).sum()

recall = num_matches/len(new_gt)
precision = (num_matches/num_edges)
print("Recall", recall)
print("Precision", precision)
print("Number of comparisons",num_edges)
cep_f1 = 2*((precision*recall)/(precision+recall))
print("F1 score",cep_f1)

Recall 1.0
Precision 0.01122334455667789
Number of comparisons 891
F1 score 0.022197558268590455


In [47]:
newColumns = ["no_of_edges","no_of_matches","retaind_edges"]
CEP_results.toDF(newColumns).show(10)

+-----------+-------------+--------------------+
|no_of_edges|no_of_matches|       retaind_edges|
+-----------+-------------+--------------------+
|         30|            0|[{0, 30, 19}, {0,...|
|         30|            0|[{2, 38, 14}, {2,...|
|         27|            0|[{4, 34, 13}, {4,...|
|         29|            0|[{6, 40, 3}, {6, ...|
|         30|            0|[{8, 34, 68}, {8,...|
|         30|            0|[{10, 57, 44}, {1...|
|         30|            0|[{14, 58, 22}, {1...|
|         30|            0|[{18, 34, 6}, {18...|
|         29|            1|[{24, 34, 369}, {...|
|         30|            1|[{28, 58, 38}, {2...|
+-----------+-------------+--------------------+
only showing top 10 rows



In [48]:
f1_score_list = [wnp_f1,rwnp_f1,wep_f1,cep_f1]
f1_score_final = max(f1_score_list)


print("\n\n\n F1_score_final",f1_score_final)

if(f1_score_final == wnp_f1):
    final_score_result = WNP_results
elif(f1_score_final == rwnp_f1):
    final_score_result = RWNP_results
elif(f1_score_final == wep_f1):
    final_score_result = WEP_results
elif(f1_score_final == cep_f1):
    final_score_result = CEP_results





 F1_score_final 0.10204081632653061


In [49]:
# lst1newColumns = ["no_of_edges","no_of_matches","retaind_edges"]
# results.toDF(newColumns).show(truncate=False)

## Collecting edges after meta-blocking
As mentioned before, the third element of the tuples returned by the meta-blocking contains the edges.


Edges are weighted according to the weight strategy provided to the meta-blocking.

In [50]:
edges = final_score_result.flatMap(lambda x: x[2])

edges.count()

186

In [51]:
newColumns = ["profile_id","neighbor_id","neighbor_weight"]
ans = edges.toDF(newColumns)
# ans.sort_values(by=['neighbor_weight'])    #show(50000)

In [52]:

ans = spark.createDataFrame(data = edges, schema = newColumns)

In [53]:
ans.show(50000)

+----------+-----------+---------------+
|profile_id|neighbor_id|neighbor_weight|
+----------+-----------+---------------+
|         0|         34|             54|
|         0|         55|             19|
|         0|         41|             48|
|         0|         47|             26|
|         0|         42|             34|
|         0|         39|             34|
|         0|         43|             39|
|         0|         46|             23|
|         0|         57|             35|
|         0|         30|             19|
|         2|         37|             12|
|         2|         56|             17|
|         2|         53|             12|
|         2|         58|             15|
|         2|         48|             10|
|         6|         45|             12|
|         6|         54|             13|
|         8|         46|             26|
|         8|         42|             48|
|         8|         41|             46|
|         8|         35|             47|
|         8|    

In [54]:
# ans.groupBy("profile_id").max("neighbor_weight").show()

In [55]:
max_by_group = (edges
  .map(lambda x: (x[0], x))  
   .reduceByKey(lambda x1, x2: max(x1, x2, key=lambda x: x[-1]))
   .values())
                
# Convmax_by_group.collect()ert to PairwiseRD
  # Take maximum of the passed arguments by the last element (key)
  # equivalent to:
  # lambda x, y: x if x[-1] > y[-1] else y

In [56]:
max_by_group.take(60)

[(0, 34, 54),
 (2, 56, 17),
 (6, 54, 13),
 (8, 34, 68),
 (10, 34, 65),
 (14, 34, 60),
 (18, 56, 17),
 (24, 34, 369),
 (28, 38, 126),
 (26, 36, 139),
 (12, 57, 25),
 (16, 56, 19),
 (20, 30, 82),
 (22, 32, 66),
 (3, 43, 65),
 (5, 57, 47),
 (11, 35, 40),
 (13, 53, 11),
 (17, 34, 55),
 (19, 45, 18),
 (21, 31, 60),
 (23, 33, 88),
 (25, 35, 325),
 (29, 39, 238),
 (1, 57, 41),
 (15, 34, 53),
 (7, 38, 19),
 (27, 37, 60),
 (9, 56, 25)]

In [57]:
edges.take(10)

[(0, 39, 34),
 (0, 34, 54),
 (0, 41, 48),
 (0, 46, 23),
 (0, 57, 35),
 (0, 47, 26),
 (0, 42, 34),
 (0, 55, 19),
 (0, 30, 19),
 (0, 43, 39)]

In [58]:
# lst1 = []
# dataCollect = ans.collect()
# for row in dataCollect:
#     lst1.append(row[0])
    
# list_set = set(lst1)
# s = list(list_set)
# s

In [59]:
# lst0 = []
# p_id = []
# n_id = []
# e = []
# d = {}

# def cal():
#     for i in s:
#         maxm = 1
#         for row in dataCollect:
#             if(i == row[0]):
#                 if(row[2] > maxm):
#                     maxm = row[2]
#                     #print(f"{row[0]}, {row[1]}, {row[2]}")
#                     p_id.append(row[0])
#                     n_id.append(row[1]-(separator_id+1))
#                     e.append(row[2])
#                     d = {"P_id": p_id, "N_id": n_id, "weight": e}
#     return(d)

# maxm = 0
# dataCollect = ans.collect()

# a = cal()
# df =  pd.DataFrame.from_dict(a)
# df

#             #lst1.append(row[0])

In [60]:
dataCollect = max_by_group.collect()
df =  pd.DataFrame(dataCollect)
df

0   1    2
0    0  34   54
1    2  56   17
2    6  54   13
3    8  34   68
4   10  34   65
5   14  34   60
6   18  56   17
7   24  34  369
8   28  38  126
9   26  36  139
10  12  57   25
11  16  56   19
12  20  30   82
13  22  32   66
14   3  43   65
15   5  57   47
16  11  35   40
17  13  53   11
18  17  34   55
19  19  45   18
20  21  31   60
21  23  33   88
22  25  35  325
23  29  39  238
24   1  57   41
25  15  34   53
26   7  38   19
27  27  37   60
28   9  56   25

In [61]:
df.rename(columns = {0:'P_id', 1:'N_id',
                              2:'weight'}, inplace = True)
df.head()

P_id  N_id  weight
0     0    34      54
1     2    56      17
2     6    54      13
3     8    34      68
4    10    34      65

In [62]:
n = df.filter(["P_id","N_id","weight"])
n = pd.DataFrame(n)
n

P_id  N_id  weight
0      0    34      54
1      2    56      17
2      6    54      13
3      8    34      68
4     10    34      65
5     14    34      60
6     18    56      17
7     24    34     369
8     28    38     126
9     26    36     139
10    12    57      25
11    16    56      19
12    20    30      82
13    22    32      66
14     3    43      65
15     5    57      47
16    11    35      40
17    13    53      11
18    17    34      55
19    19    45      18
20    21    31      60
21    23    33      88
22    25    35     325
23    29    39     238
24     1    57      41
25    15    34      53
26     7    38      19
27    27    37      60
28     9    56      25

In [63]:
avg_val = n["weight"].mean()
avg_val = round(avg_val,0)
avg_val

78.0

In [64]:
n["N_id"] = n["N_id"]-(separator_id+1)
new_df = n[(n['weight']>avg_val)]
new_df

P_id  N_id  weight
7     24     4     369
8     28     8     126
9     26     6     139
12    20     0      82
21    23     3      88
22    25     5     325
23    29     9     238

In [65]:
new_df["c_id"] = [x for x in range(0,len(new_df))]
new_df

/tmp/ipykernel_22394/1762727809.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df["c_id"] = [x for x in range(0,len(new_df))]


P_id  N_id  weight  c_id
7     24     4     369     0
8     28     8     126     1
9     26     6     139     2
12    20     0      82     3
21    23     3      88     4
22    25     5     325     5
23    29     9     238     6

In [66]:
col = ["P_id","N_id","weight","cluster_id"]
df_weight = spark.createDataFrame(new_df, schema=col)
df_weight.show()

+----+----+------+----------+
|P_id|N_id|weight|cluster_id|
+----+----+------+----------+
|  24|   4|   369|         0|
|  28|   8|   126|         1|
|  26|   6|   139|         2|
|  20|   0|    82|         3|
|  23|   3|    88|         4|
|  25|   5|   325|         5|
|  29|   9|   238|         6|
+----+----+------+----------+



In [67]:
ss = SparkSession(sc)


In [68]:
df1 = ss.read.option("header",True)\
        .option("inferSchema",True)\
        .csv('hdfs://localhost:9000/Final_Project/sample/Sample1_cleaned.csv')
df1.show(5)

+---+----------------+------------------+--------------------+---+------------+---------+-----+-------+
| id|     DisplayName|          Location|             AboutMe|Age|       phone|     City|State|Country|
+---+----------------+------------------+--------------------+---+------------+---------+-----+-------+
|  0|       Community|on the server farm|Hi, I'm not reall...| 27|655-452-4816|     null| null|   null|
|  1|    Geoff Dalgas|     Corvallis, OR|Dev #2 who helped...| 24|460-380-5210|Corvallis|   OR|   null|
|  2|             Jin|       Raleigh, NC|I design stuff fo...| 35|728-871-7285|  Raleigh|   NC|   null|
|  3|Stefan Lasiewski|      Berkeley, CA|Stefan Lasiewski....| 25|728-363-9979| Berkeley|   CA|   null|
|  4|      txwikinger|            Canada|Ubuntu member, Ku...| 35|343-135-3102|   Canada| null|   null|
+---+----------------+------------------+--------------------+---+------------+---------+-----+-------+
only showing top 5 rows



In [69]:
df2 = ss.read.option("header",True)\
        .option("inferSchema",True)\
        .csv('file:///home/aditi/BigData/AAI_Project/dataset/Sample2_cleaned.csv')
df2.show(5)

+---+----------------+--------------------+--------------------+---+------------+--------------+------------+-------+
| id|     DisplayName|            Location|             AboutMe|Age|       phone|          City|       State|Country|
+---+----------------+--------------------+--------------------+---+------------+--------------+------------+-------+
|  0|         tanteee|  Oldenburg, Germany|Just some random ...| 23|507-293-0738|     Oldenburg|     Germany|   null|
|  1|Alan Featherston|Wellington, New Z...|       Data Engineer| 29|991-246-5372|    Wellington| New Zealand|   null|
|  2|         jamespo|      United Kingdom|Just another Linu...| 38|665-081-3271|United Kingdom|        null|   null|
|  3|       fwaechter|          Vienna, VA|I'm feeling rough...| 28|923-759-1017|        Vienna|          VA|   null|
|  4|    Ceppi Marco |          Vienna, VA|I'm a Moderator -...| 32|375-304-3599|        Vienna|          VA|   null|
+---+----------------+--------------------+-------------

In [71]:
#common_df = df2.join(df_weight,df2.id_2 == df_weight.N_id,"inner")

In [72]:
# inner join on two dataframes
common_df = df1.join(df_weight,df1.id == df_weight.P_id,"inner")

In [73]:
common_df

DataFrame[id: int, DisplayName: string, Location: string, AboutMe: string, Age: int, phone: string, City: string, State: string, Country: string, P_id: bigint, N_id: bigint, weight: bigint, cluster_id: bigint]

In [74]:
pd_common_df = common_df.toPandas()
pd_common_df

id       DisplayName                         Location  \
0  24       Marco Ceppi                       Vienna, VA   
1  28          jacksonh                       Boston, MA   
2  26       Jim Fiorato  Wheaton, Illinois United States   
3  20             tante                          Germany   
4  23         fwaechter                       Vienna, VA   
5  25  Benjamin Pollack                              USA   
6  29              Dale                              USA   

                                             AboutMe  Age         phone  \
0  I'm a Moderator - if you have any issues, ques...   32  375-304-3599   
1  I am a developer on the Mono and Moonlight pro...   35  490-267-2635   
2  I'm very important. I have many leather-bound ...   35  757-448-8875   
3  Just some random coder, for more check out my ...   23  507-293-0738   
4  I'm feeling rough, I'm feeling raw, I'm in the...   28  923-759-1017   
5  I work at Bakpax, where we're focusing on maki...   30  820-002-3042   
6  I am an Electrical Engineer / Software Develop...   18  319-613-6622   

      City                    State Country  P_id  N_id  weight  cluster_id  
0   Vienna                       VA    None    24     4     369           0  
1   Boston                       MA    None    28     8     126           1  
2  Wheaton   Illinois United States    None    26     6     139           2  
3  Germany                     None    None    20     0      82           3  
4   Vienna                       VA    None    23     3      88           4  
5      USA                     None    None    25     5     325           5  
6      USA                     None    None    29     9     238           6

In [75]:
common_N = df2.join(df_weight,df2.id == df_weight.N_id,"inner")

In [76]:
#common_N.take(2)

In [77]:
pd_N = common_N.toPandas()
pd_N

id       DisplayName                         Location  \
0   4      Ceppi Marco                        Vienna, VA   
1   8          jacksonh                       Boston, MA   
2   6      Jimm Fiorato  Wheaton, Illinois United States   
3   0           tanteee               Oldenburg, Germany   
4   3         fwaechter                       Vienna, VA   
5   5  Benjamin Pollack       Raleigh, NC, United States   
6   9        Dale Ragan                    United States   

                                             AboutMe  Age         phone  \
0  I'm a Moderator - if you have any issues, ques...   32  375-304-3599   
1  I am a developer on the Mono and Moonlight pro...   35  490-267-2635   
2  I'm very important. I have many leather-bound ...   35  757-448-8875   
3  Just some random coder, for more check out my ...   23  507-293-0738   
4  I'm feeling rough, I'm feeling raw, I'm in the...   28  923-759-1017   
5  I work at Bakpax, where we're focusing on maki...   30  820-002-3042   
6  I am an Electrical Engineer / Software Develop...   18  319-613-6622   

            City                    State         Country  P_id  N_id  weight  \
0         Vienna                       VA            None    24     4     369   
1         Boston                       MA            None    28     8     126   
2        Wheaton   Illinois United States            None    26     6     139   
3      Oldenburg                  Germany            None    20     0      82   
4         Vienna                       VA            None    23     3      88   
5        Raleigh                       NC   United States    25     5     325   
6  United States                     None            None    29     9     238   

   cluster_id  
0           0  
1           1  
2           2  
3           3  
4           4  
5           5  
6           6

In [78]:
common_df_2 = common_df.join(df2,common_df.N_id == df2.id,"inner")
pd_common_df_2 = common_df_2.toPandas()
# pd_common_df_2 = pd_common_df_2.drop(["Location","Location_2","P_id","N_id"], axis=1)
# pd_common_df_2

In [79]:
#pd_common_df_2

In [80]:
#pd_common_df_2.to_csv('/home/aditi/BigData/AAI_Project/dataset/Output.csv', header=True, index=False)

In [81]:
# baseDir = 'hdfs://localhost:9000/Final_Project/sample/Output.csv'
# common_df_2.write.option("header", True).mode('overwrite').csv(baseDir)

In [82]:
common_df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- DisplayName: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- AboutMe: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- phone: string (nullable = true)
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- P_id: long (nullable = true)
 |-- N_id: long (nullable = true)
 |-- weight: long (nullable = true)
 |-- cluster_id: long (nullable = true)



In [83]:
import functools

def unionAll(dfs):
    return functools.reduce(lambda df1, df2: df1.union(df2.select(df1.columns)), dfs)
  
  
# unionAll
result3 = unionAll([common_df, common_N])
result3.show()

+---+----------------+--------------------+--------------------+---+------------+-------------+--------------------+--------------+----+----+------+----------+
| id|     DisplayName|            Location|             AboutMe|Age|       phone|         City|               State|       Country|P_id|N_id|weight|cluster_id|
+---+----------------+--------------------+--------------------+---+------------+-------------+--------------------+--------------+----+----+------+----------+
| 24|     Marco Ceppi|          Vienna, VA|I'm a Moderator -...| 32|375-304-3599|       Vienna|                  VA|          null|  24|   4|   369|         0|
| 28|        jacksonh|          Boston, MA|I am a developer ...| 35|490-267-2635|       Boston|                  MA|          null|  28|   8|   126|         1|
| 26|     Jim Fiorato|Wheaton, Illinois...|I'm very importan...| 35|757-448-8875|      Wheaton| Illinois United ...|          null|  26|   6|   139|         2|
| 20|           tante|             Germa

In [84]:
# result3 = result3.groupBy("cluster_id","id")

In [85]:
final = result3.toPandas()

In [86]:
final.columns

Index(['id', 'DisplayName', 'Location', 'AboutMe', 'Age', 'phone', 'City',
       'State', 'Country', 'P_id', 'N_id', 'weight', 'cluster_id'],
      dtype='object')

In [87]:
final = final.drop(["P_id","N_id","weight"], axis=1)
final.head()

id  DisplayName                         Location  \
0  24  Marco Ceppi                       Vienna, VA   
1  28     jacksonh                       Boston, MA   
2  26  Jim Fiorato  Wheaton, Illinois United States   
3  20        tante                          Germany   
4  23    fwaechter                       Vienna, VA   

                                             AboutMe  Age         phone  \
0  I'm a Moderator - if you have any issues, ques...   32  375-304-3599   
1  I am a developer on the Mono and Moonlight pro...   35  490-267-2635   
2  I'm very important. I have many leather-bound ...   35  757-448-8875   
3  Just some random coder, for more check out my ...   23  507-293-0738   
4  I'm feeling rough, I'm feeling raw, I'm in the...   28  923-759-1017   

      City                    State Country  cluster_id  
0   Vienna                       VA    None           0  
1   Boston                       MA    None           1  
2  Wheaton   Illinois United States    None           2  
3  Germany                     None    None           3  
4   Vienna                       VA    None           4

In [88]:
final_cluster = final.groupby(['cluster_id','id'])

In [89]:
final_cluster.first()

DisplayName                         Location  \
cluster_id id                                                      
0          4       Ceppi Marco                        Vienna, VA   
           24       Marco Ceppi                       Vienna, VA   
1          8           jacksonh                       Boston, MA   
           28          jacksonh                       Boston, MA   
2          6       Jimm Fiorato  Wheaton, Illinois United States   
           26       Jim Fiorato  Wheaton, Illinois United States   
3          0            tanteee               Oldenburg, Germany   
           20             tante                          Germany   
4          3          fwaechter                       Vienna, VA   
           23         fwaechter                       Vienna, VA   
5          5   Benjamin Pollack       Raleigh, NC, United States   
           25  Benjamin Pollack                              USA   
6          9         Dale Ragan                    United States   
           29              Dale                              USA   

                                                         AboutMe  Age  \
cluster_id id                                                           
0          4   I'm a Moderator - if you have any issues, ques...   32   
           24  I'm a Moderator - if you have any issues, ques...   32   
1          8   I am a developer on the Mono and Moonlight pro...   35   
           28  I am a developer on the Mono and Moonlight pro...   35   
2          6   I'm very important. I have many leather-bound ...   35   
           26  I'm very important. I have many leather-bound ...   35   
3          0   Just some random coder, for more check out my ...   23   
           20  Just some random coder, for more check out my ...   23   
4          3   I'm feeling rough, I'm feeling raw, I'm in the...   28   
           23  I'm feeling rough, I'm feeling raw, I'm in the...   28   
5          5   I work at Bakpax, where we're focusing on maki...   30   
           25  I work at Bakpax, where we're focusing on maki...   30   
6          9   I am an Electrical Engineer / Software Develop...   18   
           29  I am an Electrical Engineer / Software Develop...   18   

                      phone           City                    State  \
cluster_id id                                                         
0          4   375-304-3599         Vienna                       VA   
           24  375-304-3599         Vienna                       VA   
1          8   490-267-2635         Boston                       MA   
           28  490-267-2635         Boston                       MA   
2          6   757-448-8875        Wheaton   Illinois United States   
           26  757-448-8875        Wheaton   Illinois United States   
3          0   507-293-0738      Oldenburg                  Germany   
           20  507-293-0738        Germany                     None   
4          3   923-759-1017         Vienna                       VA   
           23  923-759-1017         Vienna                       VA   
5          5   820-002-3042        Raleigh                       NC   
           25  820-002-3042            USA                     None   
6          9   319-613-6622  United States                     None   
           29  319-613-6622            USA                     None   

                      Country  
cluster_id id                  
0          4             None  
           24            None  
1          8             None  
           28            None  
2          6             None  
           26            None  
3          0             None  
           20            None  
4          3             None  
           23            None  
5          5    United States  
           25            None  
6          9             None  
           29            None

In [90]:
df = pd.concat(map(lambda x: x[1], final_cluster))

In [91]:
df = df.reset_index().drop(["index","id"],axis =1)

new_col = [x for x in range(0,len(df))]
df.insert(loc=0, column='id', value=new_col)

In [92]:
df

id       DisplayName                         Location  \
0    0      Ceppi Marco                        Vienna, VA   
1    1       Marco Ceppi                       Vienna, VA   
2    2          jacksonh                       Boston, MA   
3    3          jacksonh                       Boston, MA   
4    4      Jimm Fiorato  Wheaton, Illinois United States   
5    5       Jim Fiorato  Wheaton, Illinois United States   
6    6           tanteee               Oldenburg, Germany   
7    7             tante                          Germany   
8    8         fwaechter                       Vienna, VA   
9    9         fwaechter                       Vienna, VA   
10  10  Benjamin Pollack       Raleigh, NC, United States   
11  11  Benjamin Pollack                              USA   
12  12        Dale Ragan                    United States   
13  13              Dale                              USA   

                                              AboutMe  Age         phone  \
0   I'm a Moderator - if you have any issues, ques...   32  375-304-3599   
1   I'm a Moderator - if you have any issues, ques...   32  375-304-3599   
2   I am a developer on the Mono and Moonlight pro...   35  490-267-2635   
3   I am a developer on the Mono and Moonlight pro...   35  490-267-2635   
4   I'm very important. I have many leather-bound ...   35  757-448-8875   
5   I'm very important. I have many leather-bound ...   35  757-448-8875   
6   Just some random coder, for more check out my ...   23  507-293-0738   
7   Just some random coder, for more check out my ...   23  507-293-0738   
8   I'm feeling rough, I'm feeling raw, I'm in the...   28  923-759-1017   
9   I'm feeling rough, I'm feeling raw, I'm in the...   28  923-759-1017   
10  I work at Bakpax, where we're focusing on maki...   30  820-002-3042   
11  I work at Bakpax, where we're focusing on maki...   30  820-002-3042   
12  I am an Electrical Engineer / Software Develop...   18  319-613-6622   
13  I am an Electrical Engineer / Software Develop...   18  319-613-6622   

             City                    State         Country  cluster_id  
0          Vienna                       VA            None           0  
1          Vienna                       VA            None           0  
2          Boston                       MA            None           1  
3          Boston                       MA            None           1  
4         Wheaton   Illinois United States            None           2  
5         Wheaton   Illinois United States            None           2  
6       Oldenburg                  Germany            None           3  
7         Germany                     None            None           3  
8          Vienna                       VA            None           4  
9          Vienna                       VA            None           4  
10        Raleigh                       NC   United States           5  
11            USA                     None            None           5  
12  United States                     None            None           6  
13            USA                     None            None           6

In [93]:
df.to_csv('/home/aditi/BigData/AAI_Project/dataset/output_file.csv', header=True, index=False)

######################################################################################

###############################################################################

In [94]:

#final.groupby("cluster_id")["id"].mean().reset_index()[["cluster_id", "id"]].to_csv('/home/aditi/BigData/AAI_Project/dataset/output_file.csv')